In [1]:
import nltk
import sklearn
import pandas
print("NLTK version: ", nltk.__version__)
print("Scikit-learn version: ", sklearn.__version__)
print("Pandas version: ", pandas.__version__)
import csv
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
#from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import linear_model
from scipy.sparse import hstack
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import re

NLTK version:  3.2.1
Scikit-learn version:  0.18.1
Pandas version:  0.18.1


### Loading Train and Test Tweets.

In [2]:
file_train = "train_tweets_no_accents.csv"
names = ['ID','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20','T21','T22','T23','T24','T25','T26','T27','T28','T29','T30','T31','T32','T33','T34','T35','T36','T37','T38','T39','T40','T41','T42','T43','T44','T45','T46','T47','T48','T49','T50','T51','T52','T53','T54','T55','T56','T57','T58','T59','T60','T61','T62','T63','T64','T65','T66','T67','T68','T69','T70','T71','T72','T73','T74','T75','T76','T77','T78','T79','T80','T81','T82','T83','T84','T85','T86','T87','T88','T89','T90','T91','T92','T93','T94','T95','T96','T97','T98','T99','T100']
df_train = pandas.read_csv(file_train,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

In [3]:
file_test = "test_tweets_no_accents.csv"
names = ['ID','T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20','T21','T22','T23','T24','T25','T26','T27','T28','T29','T30','T31','T32','T33','T34','T35','T36','T37','T38','T39','T40','T41','T42','T43','T44','T45','T46','T47','T48','T49','T50','T51','T52','T53','T54','T55','T56','T57','T58','T59','T60','T61','T62','T63','T64','T65','T66','T67','T68','T69','T70','T71','T72','T73','T74','T75','T76','T77','T78','T79','T80','T81','T82','T83','T84','T85','T86','T87','T88','T89','T90','T91','T92','T93','T94','T95','T96','T97','T98','T99','T100']
df_test = pandas.read_csv(file_test,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

#### We have one row per user, one column per user's tweet.
#### Train: 2800 rows, 100 columns + ID.
#### Test: 1400 rows, 100 columns + ID.

In [4]:
df_train.head(1)

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T91,T92,T93,T94,T95,T96,T97,T98,T99,T100
ID,,,,,,,,,,,,,,,,,,,,,
fff46823954870db83b3e6c74a60412c,Una practica en el Arbitraje CIADI es que si h...,@tromepe Esto me parece una falta de respeto p...,Muy buena carlincatura. https://t.co/jmYFN4rZe1,"En estos tiempos de calor infernal, ser bomber...",Estoy cansado de leer que congreso solicita co...,@Politica_ECpe @elcomercio Sres. Tengan en cla...,@DiarioAltavoz Una historia con un mismo villa...,"@DiarioAltavoz Fiscalia vs. Congreso, es una l...",@noticiAmerica Y de que manera ayudaremos si p...,@ammy_rocks24 te hablan creo Dolly. En coquete...,...,"Dicen que todos los cambios son buenos, a empe...",@noticiAmerica Hubo show para toda clase de pu...,Para algunos este discurso fue mas de lo mismo...,Excelente participacion la del congresista Fra...,"Dios te da tres respuestas: Si, no o espera. B...",Como sea queria casarse. https://t.co/qxQxs6Jlro,El mundo ya se perdio. https://t.co/04G71M0fj5,Le quedo grande el papel de Guason a Jared Let...,@Luisdiazc21 una caracteristica tuya mi buen a...,"Macho peruano que se respeta, toma chela helad..."


In [5]:
df_test.head(1)

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T91,T92,T93,T94,T95,T96,T97,T98,T99,T100
ID,,,,,,,,,,,,,,,,,,,,,
ffb5c29c835e3cecbdaa97bfea5bbe3b,Que se haga tu voluntad y no la mia Señor... S...,A congregarse!!!LEVANTATE... DEJA LAS EXCUSAS ...,EVANGELISTA JUAN CARLOS ACEVEDO PFEIFFER https...,"... La lengua, miembro pequeño del cuerpo pero...",13. Por eso en circunstancias como estas guard...,"ANTE LA IGNORANCIA, LA ESTUPIDEZ Y LA INSOLENC...",Si hay algo que nunca... jamas... never... no ...,No preguntes porque?... Deja simplemente que e...,Mi deber esta en permitir que se haga su volun...,EVANGELISTA JUAN CARLOS ACEVEDO PFEIFFER https...,...,'Can= Puedo VS. Can''t= No puedoJESUCRISTO SI ...,... En ti tengo paz!!!EVANGELISTA JUAN CARLOS ...,SOY UN GUERRERO DEL DIOS VIVIENTE Y JEHOVa DE ...,1. En ese momento los discipulos se acercaron ...,Cristo resucito... Los demas \no\!EN QUIEN CRE...,... No cometas ese error... Te arrepentiras!!!...,"13. »Por tanto —agrego Jesus—, mantenganse des...","38. Den, y se les dara: se les echara en el re...",... Dar gracias al Señor en todo tiempo!!!AMeN...,"1. Señor , oye mi justo ruego; escucha mi clam..."


### Loading Train and Test labels.

In [6]:
file_labels_train = "train_classes.csv"
names = ['ID','Variety','Gender']
df_train_labels = pandas.read_csv(file_labels_train,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

In [7]:
file_labels_test = "test_classes.csv"
names = ['ID','Variety','Gender']
df_test_labels = pandas.read_csv(file_labels_test,names=names,sep='\t',encoding='utf-8',header=None,index_col=0,quoting=csv.QUOTE_NONE)

In [8]:
df_train_labels.head(1)

,Variety,Gender
ID,,
fff46823954870db83b3e6c74a60412c,peru,male


In [9]:
df_test_labels.head(1)

,Variety,Gender
ID,,
ffb5c29c835e3cecbdaa97bfea5bbe3b,colombia,male


### Tweets' text preocessing. tolowercase, Twitter tokenizer tokenization and spanish stop-words romoval.

In [10]:
tknzr = TweetTokenizer()

#### to.lower() all the text.

In [11]:
df_train = df_train.applymap(lambda x: x.lower())

In [12]:
df_test = df_test.applymap(lambda x: x.lower())

#### We build a list to access every tweet per user.

In [13]:
lista_tweets = ['T1','T2','T3','T4','T5','T6','T7','T8','T9','T10','T11','T12','T13','T14','T15','T16','T17','T18','T19','T20','T21','T22','T23','T24','T25','T26','T27','T28','T29','T30','T31','T32','T33','T34','T35','T36','T37','T38','T39','T40','T41','T42','T43','T44','T45','T46','T47','T48','T49','T50','T51','T52','T53','T54','T55','T56','T57','T58','T59','T60','T61','T62','T63','T64','T65','T66','T67','T68','T69','T70','T71','T72','T73','T74','T75','T76','T77','T78','T79','T80','T81','T82','T83','T84','T85','T86','T87','T88','T89','T90','T91','T92','T93','T94','T95','T96','T97','T98','T99','T100']

#### Punctuation removal.

In [14]:
for item in lista_tweets:
    df_train[item]=df_train[item].str.replace('[^\w\s]','')

In [15]:
for item in lista_tweets:
    df_test[item]=df_test[item].str.replace('[^\w\s]','')

#### Tokenization.

In [16]:
for item in lista_tweets:
    df_train[item] = df_train.apply(lambda row: tknzr.tokenize(row[item]),axis=1)

In [17]:
for item in lista_tweets:
    df_test[item] = df_test.apply(lambda row: tknzr.tokenize(row[item]),axis=1)

#### Stop-words removal.

In [18]:
stop = stopwords.words('spanish')

In [19]:
for item in lista_tweets:
    df_train[item] = df_train[item].apply(lambda tweet: [word for word in tweet if word not in stop])

In [20]:
for item in lista_tweets:
    df_test[item] = df_test[item].apply(lambda tweet: [word for word in tweet if word not in stop])

#### Stem.

In [17]:
stemmer = SnowballStemmer('spanish')

In [18]:
for item in lista_tweets:
    df_train[item] = df_train[item].apply(lambda tweet: [stemmer.stem(w) for w in tweet])

In [20]:
for item in lista_tweets:
    df_test[item] = df_test[item].apply(lambda tweet: [stemmer.stem(w) for w in tweet])

#### Packing tweets together.

In [21]:
for item in lista_tweets:
    df_train[item] = df_train[item].apply(lambda x: " ".join(str(z) for z in x))

In [22]:
for item in lista_tweets:
    df_test[item] = df_test[item].apply(lambda x: " ".join(str(z) for z in x))

## Modeling and evaluating.

### Loading labels.

In [23]:
label_var_train = df_train_labels["Variety"].values.tolist()
label_gender_train = df_train_labels["Gender"].values.tolist()

In [24]:
label_var_test = df_test_labels["Variety"].values.tolist()
label_gender_test = df_test_labels["Gender"].values.tolist()

### Grouping all tweets per user (train and test)

In [25]:
tweets_train = []
for i in range(len(df_train)):
    aux = df_train.iloc[i].values.tolist()
    tweets_train.append(' '.join(aux))

In [26]:
tweets_test = []
for i in range(len(df_test)):
    aux = df_test.iloc[i].values.tolist()
    tweets_test.append(' '.join(aux))

### Tf-Idf Unigrams + corpus stop-words removal (max_df=0.90)

In [27]:
TfIdf_uni_df=TfidfVectorizer(analyzer="word",lowercase=False, ngram_range=(1,1), max_df=0.90, min_df=0)

In [28]:
train_tfidf_uni_df = TfIdf_uni_df.fit_transform(tweets_train)

In [29]:
test_tfidf_uni_df = TfIdf_uni_df.transform(tweets_test)

#### Trainning and Predicting using Muti-Layer Perceptron

In [30]:
ann_var = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,), random_state=1)

In [31]:
ann_var.fit(train_tfidf_uni_df, label_var_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [32]:
pred_ann_var = ann_var.predict(test_tfidf_uni_df)

In [33]:
acc_tfidf_var_ann = accuracy_score(label_var_test, pred_ann_var)
f1_tfidf_var_ann = metrics.f1_score(label_var_test, pred_ann_var, average='weighted')

In [34]:
print ("ANN Variety Accuracy amb features + Tf-Idf= ",str(acc_tfidf_var_ann))
print ("ANN Variety f1-score amb features + Tf-Idf= ",str(f1_tfidf_var_ann))

ANN Variety Accuracy amb features + Tf-Idf=  0.917857142857
ANN Variety f1-score amb features + Tf-Idf=  0.917851169257
